In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
titanic_data = pd.read_csv('titanic_data.csv', index_col=0)
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def num_missing_values(series):
    bool_series = pd.isnull(series)
    return bool_series.sum()

In [4]:
print('Column', ' Number of missing values')
print('Survived', num_missing_values(titanic_data['Survived']))
print('Pclass', num_missing_values(titanic_data['Pclass']))
print('Name', num_missing_values(titanic_data['Name']))
print('Sex', num_missing_values(titanic_data['Sex']))
print('Age', num_missing_values(titanic_data['Age']))
print('SibSp', num_missing_values(titanic_data['SibSp']))
print('Parch', num_missing_values(titanic_data['Parch']))
print('Ticket', num_missing_values(titanic_data['Ticket']))
print('Fare', num_missing_values(titanic_data['Fare']))
print('Cabin', num_missing_values(titanic_data['Cabin']))
print('Embarked', num_missing_values(titanic_data['Embarked']))

Column  Number of missing values
Survived 0
Pclass 0
Name 0
Sex 0
Age 177
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 687
Embarked 2


In [5]:
#checking whether there is a person recorded more than once, and whether there is one ticket per person
print(titanic_data['Ticket'].is_unique)
print(titanic_data['Name'].is_unique)

False
True


In [6]:
bool_duplicate_tickets = titanic_data.duplicated(subset='Ticket', keep=False) #creating a boolean list to check the number of non-unique tickets
bool_duplicate_tickets.sum() 

344

In [7]:
#since the number of non-unique tickets was significant, I created a dataset with just those tickets to invesigate it
duplicate_tickets = titanic_data[bool_duplicate_tickets]
duplicate_tickets.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S


In [8]:
"""grouping this new datasets with duplicate tickets 
and filling in some of the missing values in cabin according to the ticket they have
i.e. if two people have the same ticket and for one of them Cabin information is missing 
then this will fill it with the Cabin information from the other passenger""" 
pd.options.mode.chained_assignment = None 
duplicate_tickets.Cabin = duplicate_tickets.groupby('Ticket').Cabin.transform(lambda x: x.bfill().ffill())
titanic_data.Cabin = titanic_data.Cabin.fillna(value = duplicate_tickets.Cabin)
num_missing_values(titanic_data['Cabin'])

676

In [9]:
#Changing the cabin values to show only whether passengers had a cabin or not
titanic_data['Cabin']=titanic_data['Cabin'].fillna(value = 0)
titanic_data.loc[titanic_data.Cabin !=0, 'Cabin'] = 1

In [10]:
len(titanic_data['Cabin']) - titanic_data['Cabin'].sum()

676

In [11]:
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,1,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,1,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S


In [12]:
titanic_data.loc[titanic_data['Fare'].idxmax()]

Survived                   1
Pclass                     1
Name        Ward, Miss. Anna
Sex                   female
Age                       35
SibSp                      0
Parch                      0
Ticket              PC 17755
Fare                 512.329
Cabin                      1
Embarked                   C
Name: 259, dtype: object

In [13]:
titanic_data['Fare'].nlargest(10)

PassengerId
259    512.3292
680    512.3292
738    512.3292
28     263.0000
89     263.0000
342    263.0000
439    263.0000
312    262.3750
743    262.3750
119    247.5208
Name: Fare, dtype: float64

In [14]:
"""Since the largest Fare value was almost twice the next smaller one, 
and after printing out the the passengers with the same fare values, 
I noticed they had the same ticket, so it is reasonable to assume that 
the Fare value was the price of the Ticket no matter for how many passengers it was meant"""
titanic_data.loc[[28,89,342,439]]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,1,S
89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0,1,S
342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0,1,S
439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0,1,S


In [15]:
"""I wanted to know the Ticket price per passenger, so I grouped by 
Ticket and divided the Fare for each passenger 
by the number of passengers in each ticket group"""
titanic_data.Fare = titanic_data.groupby('Ticket').Fare.apply(lambda x: x / len(x))

In [16]:
#Checking whether the method worked with some of the passengers that had highest Fare previously
print(titanic_data['Fare'].nlargest(10))
print(titanic_data.loc[[28,89,342,439]])

PassengerId
528    221.7792
378    211.5000
259    170.7764
680    170.7764
738    170.7764
312    131.1875
743    131.1875
119    123.7604
300    123.7604
836     83.1583
Name: Fare, dtype: float64
             Survived  Pclass                            Name     Sex   Age  \
PassengerId                                                                   
28                  0       1  Fortune, Mr. Charles Alexander    male  19.0   
89                  1       1      Fortune, Miss. Mabel Helen  female  23.0   
342                 1       1  Fortune, Miss. Alice Elizabeth  female  24.0   
439                 0       1               Fortune, Mr. Mark    male  64.0   

             SibSp  Parch Ticket   Fare  Cabin Embarked  
PassengerId                                              
28               3      2  19950  65.75      1        S  
89               3      2  19950  65.75      1        S  
342              3      2  19950  65.75      1        S  
439              1      4  19950  65

In [17]:
titanic_data['Fare'].nlargest(10)

PassengerId
528    221.7792
378    211.5000
259    170.7764
680    170.7764
738    170.7764
312    131.1875
743    131.1875
119    123.7604
300    123.7604
836     83.1583
Name: Fare, dtype: float64

In [18]:
print(titanic_data.loc[[528,378,259,680,738,312,743,119,300]]) # Seemes more reasonable compared the the previous values

             Survived  Pclass  \
PassengerId                     
528                 0       1   
378                 0       1   
259                 1       1   
680                 1       1   
738                 1       1   
312                 1       1   
743                 1       1   
119                 0       1   
300                 1       1   

                                                        Name     Sex   Age  \
PassengerId                                                                  
528                                       Farthing, Mr. John    male   NaN   
378                                Widener, Mr. Harry Elkins    male  27.0   
259                                         Ward, Miss. Anna  female  35.0   
680                       Cardeza, Mr. Thomas Drake Martinez    male  36.0   
738                                   Lesurer, Mr. Gustave J    male  35.0   
312                               Ryerson, Miss. Emily Borie  female  18.0   
743         

In [19]:
titanic_data['Fare'].nsmallest(20)

PassengerId
180    0.000000
264    0.000000
272    0.000000
278    0.000000
303    0.000000
414    0.000000
467    0.000000
482    0.000000
598    0.000000
634    0.000000
675    0.000000
733    0.000000
807    0.000000
816    0.000000
823    0.000000
9      3.711100
173    3.711100
870    3.711100
379    4.012500
14     4.467857
Name: Fare, dtype: float64

In [20]:
titanic_data.loc[[180,264,272,278,303,414,467,482,598,634,675,733,807,816,823]]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,0,S
264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,1,S
272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,0,S
278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,0,S
303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,0,S
414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,0,S
467,0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,0,S
482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,0,S
598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,0,S


In [21]:
"""Fare value of 0 means passengers got onto Titanic for free,
and some of them even had 1st class privileges, but given that I
have no information that free tickets were given out I can only
assume that this was mistake in the data, and because they are only 
15 passengers(<0.25%) out of dataset of more than 800 I decided to drop them""" 
titanic_data.drop([180,264,272,278,303,414,467,482,598,634,675,733,807,816,823],inplace=True)
titanic_data['Fare'].nsmallest(20)

PassengerId
9      3.711100
173    3.711100
870    3.711100
379    4.012500
14     4.467857
120    4.467857
542    4.467857
543    4.467857
611    4.467857
814    4.467857
851    4.467857
139    4.608350
64     4.650000
168    4.650000
361    4.650000
635    4.650000
643    4.650000
820    4.650000
449    4.814575
470    4.814575
Name: Fare, dtype: float64

In [22]:
titanic_data['Fare'].describe()

count    876.000000
mean      18.093595
std       21.269940
min        3.711100
25%        7.775000
50%        9.225000
75%       25.671875
max      221.779200
Name: Fare, dtype: float64

In [23]:
"""Grouping by port and exploring the differences
in Fare, Age according to the port the price in port
C seemes to be much higher than the other 2, while
there is no significant difference in age mean"""

grouped_by_port = titanic_data.groupby('Embarked')
print(grouped_by_port.Age.apply(num_missing_values))
print(grouped_by_port.Age.mean())
print(grouped_by_port.Fare.mean())
print(grouped_by_port.size())

Embarked
C    38
Q    49
S    82
Name: Age, dtype: int64
Embarked
C    30.814769
Q    28.089286
S    29.372486
Name: Age, dtype: float64
Embarked
C    31.989172
Q     9.061095
S    15.418289
Name: Fare, dtype: float64
Embarked
C    168
Q     77
S    629
dtype: int64


In [24]:
"""after grouping by port and class it can be seen 
why the price in port C is way higher(more than 50% of passengers are 1st class), 
and why the price in port Q is so low(72 out of 78 passengers are 3rd class)"""
port_class = titanic_data.groupby(['Embarked', 'Pclass'])
print(port_class.size())

Embarked  Pclass
C         1          85
          2          17
          3          66
Q         1           2
          2           3
          3          72
S         1         122
          2         158
          3         349
dtype: int64


In [25]:
group_class = titanic_data.groupby('Pclass')
group_class['Fare'].mean() #results are as expected, first class tickets are way more expensive than the other 2

Pclass
1    44.684716
2    13.771676
3     8.152271
Name: Fare, dtype: float64

In [26]:
group_by_sex = titanic_data.groupby('Sex')
print(group_class['Age'].mean())
print(group_by_sex['Age'].mean())
print(group_by_sex.size())
group_by_sex_n_class = titanic_data.groupby(['Sex', 'Pclass'])
print(group_by_sex_n_class['Age'].mean())
print(group_by_sex_n_class.size())

Pclass
1    38.220874
2    29.877630
3    25.059601
Name: Age, dtype: float64
Sex
female    27.915709
male      30.657332
Name: Age, dtype: float64
Sex
female    314
male      562
dtype: int64
Sex     Pclass
female  1         34.611765
        2         28.722973
        3         21.750000
male    1         41.351224
        2         30.740707
        3         26.415341
Name: Age, dtype: float64
Sex     Pclass
female  1          94
        2          76
        3         144
male    1         117
        2         102
        3         343
dtype: int64


In [27]:
"""Since there is significant difference between 
the mean age of the groups when grouped by Class and Age 
(ex. 1st class male is way older than 2nd class male or 1st class female etc.), 
I decided to fill the missing values in age by the mean value in each group 
i.e. if a passenger is female and 2nd class than the missing value fro age 
will be filled with the average of this group"""

group_by_sex_n_class.Age = group_by_sex_n_class.Age.transform(lambda x: x.fillna(x.mean()))
titanic_data.Age = titanic_data.Age.fillna(value = group_by_sex_n_class.Age)
num_missing_values(titanic_data['Age'])

0

In [28]:
#converting all the age values in integers 
titanic_data['Age'] = titanic_data['Age'].astype(int)
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,0,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,1,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,0,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,26.5500,1,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,0,S


In [29]:
titanic_data[titanic_data['Embarked'].isnull()] # looking at the last missing values in the dataset

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38,0,0,113572,40.0,1,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62,0,0,113572,40.0,1,NaN


In [30]:
#Given that 73% of all passenger embarked at S, and 60% of first class passengers embarked at S I will fill these values with S
titanic_data.Embarked = titanic_data.Embarked.fillna(value = 'S')
num_missing_values(titanic_data.Embarked)

0

In [37]:
"""8 siblings and a spouse is a bit suspicious, 
but because I don't have any additional information 
I decided to leave it like that"""
titanic_data.describe() #there doesn't seem to be any weird values regarding in any of the columns 

,Survived,Pclass,Age,SibSp,Parch,Fare
count,876.000000,876.000000,876.000000,876.000000,876.000000,876.000000
mean,0.389269,2.315068,29.111872,0.531963,0.388128,18.093595
std,0.487863,0.835663,13.378663,1.110009,0.811374,21.269940
min,0.000000,1.000000,0.000000,0.000000,0.000000,3.711100
25%,0.000000,2.000000,21.000000,0.000000,0.000000,7.775000
50%,0.000000,3.000000,26.000000,0.000000,0.000000,9.225000
75%,1.000000,3.000000,36.000000,1.000000,0.000000,25.671875
max,1.000000,3.000000,80.000000,8.000000,6.000000,221.779200


In [32]:
print('Column', ' Number of missing values')
print('Survived', num_missing_values(titanic_data['Survived']))
print('Pclass', num_missing_values(titanic_data['Pclass']))
print('Name', num_missing_values(titanic_data['Name']))
print('Sex', num_missing_values(titanic_data['Sex']))
print('Age', num_missing_values(titanic_data['Age']))
print('SibSp', num_missing_values(titanic_data['SibSp']))
print('Parch', num_missing_values(titanic_data['Parch']))
print('Ticket', num_missing_values(titanic_data['Ticket']))
print('Fare', num_missing_values(titanic_data['Fare']))
print('Cabin', num_missing_values(titanic_data['Cabin']))
print('Embarked', num_missing_values(titanic_data['Embarked']))

Column  Number of missing values
Survived 0
Pclass 0
Name 0
Sex 0
Age 0
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 0
Embarked 0


In [33]:
#dropping columns that I won't be exploring in my analysis 
titanic_data.drop('Name',axis=1, inplace = True) #dropping the name column because each passenger has unique id
titanic_data.drop('Ticket', axis=1, inplace = True) #Used it up until now to clean and fix the data, dropping it becuse I won't use it in my analysis

In [34]:
#dropping some columns because of high correlation(|corr|>0.75) with ones that I'm more interested in
print(titanic_data['Pclass'].corr(titanic_data['Cabin']))
titanic_data.drop('Cabin', axis=1, inplace = True) # Dropping the cabin column because it is highly correlated to the class of the passengers, so it won't provide any additional info

-0.7524862725660322


In [41]:
"""All of the other correlations bellow are pretty low,
except for the one between Class and Fare, but it is 
still not high enough to be dropped |-0.672|<0.75, 
which I set as a threshold for dropping colums"""

print(titanic_data['Pclass'].corr(titanic_data['SibSp']))
print(titanic_data['SibSp'].corr(titanic_data['Parch']))
print(titanic_data['Age'].corr(titanic_data['SibSp']))
print(titanic_data['Age'].corr(titanic_data['Parch']))
print(titanic_data['Pclass'].corr(titanic_data['Fare']))


0.08030375122295358
0.4125767299094969
-0.24996917883360298
-0.1769845721222501
-0.6720478374876493


In [36]:
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22,1,0,7.2500,S
2,1,1,female,38,1,0,71.2833,C
3,1,3,female,26,0,0,7.9250,S
4,1,1,female,35,1,0,26.5500,S
5,0,3,male,35,0,0,8.0500,S


In [42]:
titanic_data.to_csv('cleaned_titanic_data.csv')